In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
%%capture
# dhash used for shot separation / frame skipping
!pip install dhash

# module 'scipy.misc' has no attribute 'imread' => downgrade scipy
!pip3 install scipy==1.1.0

# ValueError: Object arrays cannot be loaded when allow_pickle=False
# This version of numpy has the default value of allow_pickle as True.
!pip install numpy==1.16.1
import numpy as np

In [0]:
!git clone https://github.com/simonedeldeo/CyclicGen.git

Cloning into 'CyclicGen'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 147 (delta 5), reused 0 (delta 0), pack-reused 133
Receiving objects: 100% (147/147), 501.39 KiB | 1.42 MiB/s, done.
Resolving deltas: 100% (75/75), done.


In [0]:
%cd CyclicGen/

/content/CyclicGen


In [0]:
# creaz dataset

!rm -rf sep_eps
!mkdir data_list

import cv2
import dhash
from PIL import Image
import os
import shutil

def get_num_bits_different(hash1, hash2):
    return bin(hash1 ^ hash2).count('1')

vids = ['at_s01e01.mp4', 'at_s01e02.mp4', 'at_s01e03.mp4', 'at_s01e04.mp4', 'sao_e01.mkv', 'Akira.mkv']

for vid_path in vids:

  cap = cv2.VideoCapture('/content/drive/My Drive/__progetto int sys/data/' + vid_path)
  vid_name = vid_path.split('.')[0]
  totalFrames = cap.get(cv2.CAP_PROP_FRAME_COUNT)
  if totalFrames > 70000:
    totalFrames = 70000
  main_dir = 'sep_eps/' + vid_name
  if os.path.exists(main_dir):
      shutil.rmtree(main_dir)
  os.makedirs(main_dir)

  # 0 - 2: frame ripetuto => non lo salvo
  # 0 - 9 per Akira
  thresh_rip = 2
  if vid_name == 'Akira':
    thresh_rip = 9

  id_scena = 1
  ret, prec = cap.read()
  path = main_dir + '/' + str(id_scena).zfill(3)
  if not os.path.exists(path):
      os.mkdir(path)
  cv2.imwrite(path + '/0.jpg', prec)
  x = int(prec.shape[0] / 5)
  y = int(prec.shape[1] / 5)
  for i in range(1, int(totalFrames)):
      try:
          ret, frame = cap.read()
          prec_res = cv2.resize(prec, (y, x))
          frame_res = cv2.resize(frame, (y, x))
          h1 = dhash.dhash_int(Image.fromarray(prec_res), size=16)
          h2 = dhash.dhash_int(Image.fromarray(frame_res), size=16)

          bit_diff = get_num_bits_different(h1, h2)

          if bit_diff > thresh_rip:
              # threshold trovato sperimentalmente per sep shot = 213
              # usando 205, i falsi positivi aumentavano solo di 5
              # meglio qualche falso pos in piu' che un falso neg
              if bit_diff > 204:
                  id_scena += 1
              path = main_dir + '/' + str(id_scena).zfill(3)
              if not os.path.exists(path):
                  os.mkdir(path)
              frame = cv2.resize(frame, (1280, 720))
              cv2.imwrite(path + '/' + str(i) + '.jpg', frame)

          prec = frame

      except:
          print('frame finale: ', i)
  print(vid_name)

  #########################################################

  ep_path = 'sep_eps/' + vid_name
  scenes = os.listdir(ep_path)
  scenes.sort()

  da_canc = []
  for s in scenes:
      folder = os.path.join(ep_path, s)
      frames_number = os.listdir(folder)
      if len(frames_number)<3:
          da_canc.append(folder)

  for f in da_canc:
      shutil.rmtree(f)

at_s01e01
at_s01e02
at_s01e03
at_s01e04
frame finale:  68573
frame finale:  68574
sao_e01
Akira


In [0]:
!du -hs sep_eps

23G	sep_eps


In [0]:
import os

dirs = os.listdir('sep_eps')
triplets = []
for main_dir in dirs:
  lista_dir = os.listdir(os.path.join('sep_eps', main_dir))
  lista_dir.sort()
  for d in lista_dir:
    dir_path = os.path.join('sep_eps', main_dir, d)
    toAppend = os.listdir(dir_path)
    if len(toAppend)>2:
      files = []
      for f in toAppend:
        files.append(os.path.join(dir_path, f))
      files.sort()
      for i in range(len(files)-2):
        triplets.append([files[i], files[i+1], files[i+2]])
      
print(len(triplets))

129898


In [0]:
for t in triplets:
  for i in range(1, 4):
    path = 'data_list/ucf101_train_files_frame' + str(i) + '.txt'
    with open(path, 'a') as fp:
      fp.write(t[i-1] + '\n')

In [0]:
!cp '/content/drive/My Drive/__progetto int sys/ckps/CyclicGen/ckpt_and_hed_model.zip' '/content/CyclicGen'
!unzip ckpt_and_hed_model.zip

Archive:  ckpt_and_hed_model.zip
   creating: ckpt/
   creating: ckpt/CyclicGen/
  inflating: ckpt/CyclicGen/model.meta  
  inflating: ckpt/CyclicGen/model.index  
  inflating: ckpt/CyclicGen/model.data-00000-of-00001  
   creating: ckpt/CyclicGen_large/
  inflating: ckpt/CyclicGen_large/model.meta  
  inflating: ckpt/CyclicGen_large/model.index  
  inflating: ckpt/CyclicGen_large/model.data-00000-of-00001  
   creating: hed_model/
  inflating: hed_model/new-model.ckpt.meta  
  inflating: hed_model/new-model.ckpt.data-00000-of-00001  
  inflating: hed_model/new-model.ckpt.index  
  inflating: vgg16.npy               


In [0]:
!date
!python3 CyclicGen_train_stage1.py --subset=train --train_dir='/content/drive/My Drive/__progetto int sys/ckps/CyclicGen/_new/prv_1ep720_s1' --pretrained_model_checkpoint_path=./ckpt/CyclicGen/model --max_steps=20826 | tee '/content/drive/My Drive/__progetto int sys/ckps/CyclicGen/_new/prv_1ep720_s1.txt'
!date

Wed Feb  5 12:16:22 UTC 2020
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use `tf.data.experimental.shuffle_and_repeat(...)`.
Instructions for updating:
Use `tf.data.Dataset.shuffle(buffer_size, seed)` followed by `tf.data.Dataset.repeat(count)`. Static tf.data optimizations will take care of using the fused implementation.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

2020-02-05 12:16:34.647554: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-02-05 12:16:34.689283: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:983] succ